In [1]:
import numpy as np 
import pandas as pd 
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 忽视警告
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

from tqdm import tqdm_notebook
import datetime
import time
import random
from joblib import Parallel, delayed

# 要用的模型，以及一些预处理的方法
from tensorflow import keras
from catboost import Pool, CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV

In [2]:
# 数据导入
train_X_0 = pd.read_csv("../input/feature1/train_X_features_865.csv")
train_X_1 = pd.read_csv("../input/feature2/train_X_features_865_1.csv")
y_0 = pd.read_csv("../input/feature1/train_y.csv", index_col=False,  header=None)
y_1 = pd.read_csv("../input/feature2/train_y1.csv", index_col=False,  header=None)
train_X = pd.concat([train_X_0, train_X_1], axis=0)
train_X = train_X.reset_index(drop=True)
train_X.head()

,FFT_Mag_01q0,FFT_Mag_10q0,FFT_Mag_90q0,FFT_Mag_99q0,FFT_Mag_mean0,FFT_Mag_std0,FFT_Mag_max0,FFT_Phz_mean0,FFT_Phz_std0,FFT_Mag_01q2500,FFT_Mag_10q2500,FFT_Mag_90q2500,FFT_Mag_99q2500,FFT_Mag_mean2500,FFT_Mag_std2500,FFT_Mag_max2500,FFT_Phz_mean2500,FFT_Phz_std2500,FFT_Mag_01q5000,FFT_Mag_10q5000,FFT_Mag_90q5000,FFT_Mag_99q5000,FFT_Mag_mean5000,FFT_Mag_std5000,FFT_Mag_max5000,FFT_Phz_mean5000,FFT_Phz_std5000,FFT_Mag_01q7500,FFT_Mag_10q7500,FFT_Mag_90q7500,FFT_Mag_99q7500,FFT_Mag_mean7500,FFT_Mag_std7500,FFT_Mag_max7500,FFT_Phz_mean7500,FFT_Phz_std7500,FFT_Mag_01q10000,FFT_Mag_10q10000,FFT_Mag_90q10000,FFT_Mag_99q10000,...,q01_roll_std_100,q05_roll_std_100,q95_roll_std_100,q99_roll_std_100,av_change_abs_roll_std_100,av_change_rate_roll_std_100,abs_max_roll_std_100,ave_roll_mean_100,std_roll_mean_100,max_roll_mean_100,min_roll_mean_100,q01_roll_mean_100,q05_roll_mean_100,q95_roll_mean_100,q99_roll_mean_100,av_change_abs_roll_mean_100,av_change_rate_roll_mean_100,abs_max_roll_mean_100,ave_roll_std_1000,std_roll_std_1000,max_roll_std_1000,min_roll_std_1000,q01_roll_std_1000,q05_roll_std_1000,q95_roll_std_1000,q99_roll_std_1000,av_change_abs_roll_std_1000,av_change_rate_roll_std_1000,abs_max_roll_std_1000,ave_roll_mean_1000,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
0,139.522396,409.718152,2599.097192,4061.567699,1345.706663,938.175660,11969.918774,0.022472,0.913523,237.008292,731.742174,4249.831881,6874.568414,2317.422748,1425.766416,8646.302978,-0.016085,0.907828,448.854260,1338.451624,6620.832265,9452.082006,3818.343685,2036.886799,13104.251576,0.006786,0.897903,566.136528,1828.685542,11398.359936,18595.373516,5886.087254,3894.189563,23532.881664,-0.036087,0.909921,256.880152,933.998090,5958.916634,10835.791291,...,2.275451,2.442780,8.526104,18.892797,0.000020,74941.769634,50.909740,4.618451,0.436482,12.19,-2.40,3.62,3.93,5.29,5.66,5.203469e-06,74938.678297,12.19,4.209590,2.974309,28.705276,2.502630,2.607830,2.672366,10.317476,14.107140,1.352170e-05,74563.128945,28.705276,4.618982,0.250223,5.430,3.926,4.034,4.201,5.028,5.195,4.080537e-06,74563.065284,5.430
1,103.006025,432.256164,3894.194205,8899.077054,1778.081264,1753.736076,13008.467215,-0.016746,0.913844,265.294292,1096.751852,7355.981733,11124.480251,3806.170646,2546.431723,17024.790802,-0.003393,0.920536,647.666936,2056.427540,11181.052161,16426.982660,6207.629093,3586.244884,23566.293654,-0.001122,0.908535,670.834218,2226.225751,15177.013868,25823.792387,7711.334054,5419.906778,35981.079671,-0.006243,0.922031,332.465194,1068.575688,8027.668012,14524.229995,...,2.322834,2.510906,13.634641,30.138576,0.000033,74709.627147,52.708284,4.325410,0.513053,11.78,-2.70,2.99,3.62,5.00,5.57,3.802535e-06,74705.618281,11.78,5.414893,4.472418,29.594507,2.548829,2.664467,2.760705,16.239974,23.913015,-9.045880e-05,74208.764935,29.594507,4.325288,0.207712,5.216,3.612,3.812,3.990,4.663,4.827,-7.516779e-07,74208.713592,5.216
2,121.840541,354.132240,1812.762440,2953.824113,1030.634121,684.285591,10665.922219,-0.033928,0.928496,235.951723,714.632635,3486.537639,5040.008929,1975.605682,1095.447506,7816.698187,-0.005391,0.917317,491.740705,1354.334138,6334.734457,9033.277200,3670.042914,1931.795674,12545.320475,-0.007111,0.903919,435.187536,1456.769253,9188.303499,14115.878775,4780.926331,3099.433090,19756.115049,-0.006067,0.904687,246.821665,796.153576,4675.630875,7543.952280,...,2.256304,2.409472,8.253215,15.107173,-0.000033,74872.665159,34.318383,4.310323,0.401061,7.06,1.85,3.37,3.68,4.96,5.27,-8.672448e-07,74863.629626,7.06,3.922947,2.161912,16.474262,2.456007,2.581889,2.650754,8.401994,14.176844,-7.340644e-06,74532.874909,16.474262,4.310783,0.238572,5.049,3.676,3.811,3.921,4.707,4.835,-1.174497e-06,74532.222230,5.049
3,101.571556,334.674233,1639.884609,2581.813048,964.089147,587.651030,9512.216925,0.014567,0.903097,176.859

In [3]:
#得到y数据
y = pd.concat([y_0, y_1], axis=0)
y = y.reset_index(drop=True)
y[0].shape

(33000,)

In [4]:
train_y = pd.Series(y[0].values)

In [5]:
test_X = pd.read_csv("../input/feature2/test_X_features_10.csv")
test_X.shape

(2624, 866)

In [6]:
#标准化数据
scaler = StandardScaler()
train_columns = train_X.columns

train_X[train_columns] = scaler.fit_transform(train_X[train_columns])
test_X[train_columns] = scaler.transform(test_X[train_columns])

In [7]:
# 设置交叉验证的次数
train_columns = train_X.columns
n_fold = 5

# Neural Network

In [8]:
# 定义神经网络
def create_model(input_dim=10):
    # 神经网络的序列模型, dense为神经元，dropout用来正则化
    model = keras.Sequential()
    # 第一层256个神经元
    model.add(keras.layers.Dense(256, activation="relu", input_dim=input_dim))
    model.add(keras.layers.Dropout(0.4))
    # 第二层128个神经元
    model.add(keras.layers.Dense(128, activation="relu"))
    model.add(keras.layers.Dropout(0.3))
    # 第三96个神经元
    model.add(keras.layers.Dense(96, activation="relu"))
    model.add(keras.layers.Dropout(0.3))
    # 最后一层1个神经元(只有一个time_to_failure作为结果)
    model.add(keras.layers.Dense(1, activation="linear"))
    #使用adam作为优化器,其初始学习率为0.001
    optimizer = keras.optimizers.Adam(lr=0.001) #'rmsprop'
    # 评判指标为mae
    model.compile(optimizer=optimizer,loss='mae')
    return model

# 设置EarlyStop为回调函数，监督验证集的损失变换，如果40次loss没有下降则结束
patience = 40
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, restore_best_weights=True)

In [9]:
%%time
# n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
# 将每折预测结果保存到NN_oof里面，为了后面stacking
NN_oof = np.zeros(len(train_X))

NN_predictions = np.zeros(len(test_X))

# 进行5折交叉验证
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    # 根据交叉验证得到训练集，测试集的X和y
    X_tr, X_val = train_X[train_columns].iloc[trn_idx], train_X[train_columns].iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    
    # 构造模型
    model = create_model(train_X.shape[-1])
    
    # 迭代500个epoch，每个epoch的batch大小为64
    model.fit(X_tr, y_tr, epochs=500, batch_size=64, verbose=1, callbacks=[call_ES,], validation_data=[X_val, y_val]) #
    
    # 得到NN的oof
    NN_oof[val_idx] = model.predict(X_val)[:,0]
    # 对NN预测结果取平均
    NN_predictions += model.predict(test_X[train_columns])[:,0] / folds.n_splits
    history = model.history.history
    tr_loss = history["loss"]
    val_loss = history["val_loss"]
    print(f"loss: {tr_loss[-patience-1]:.3f} | val_loss: {val_loss[-patience-1]:.3f}")
#     break
 
# 得到cv_score的成绩
cv_score = mean_absolute_error(train_y, NN_oof)


fold 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 26400 samples, validate on 6600 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
26400/26400 [==============================] - 6s 242us/sample - loss: 2.5035 - val_loss: 2.3079
Epoch 2/300
26400/26400 [==============================] - 3s 108us/sample - loss: 2.2374 - val_loss: 2.3322
Epoch 3/300
26400/26400 [==============================] - 3s 108us/sample - loss: 2.1800 - val_loss: 2.1108
Epoch 4/300
26400/26400 [==============================] - 3s 107us/sample - loss: 2.1172 - val_loss: 2.0718
Epoch 5/300
26400/26400 [==============================] - 3s 109us/sample - loss: 2.0740 - val_loss: 2.0010
Epoch 6/300
26400/26400 [==============================] - 3s 107us/sample - loss: 2.0500 - val_loss: 2.0435
Epoch 7/300
26400/26400 [==============================

Epoch 67/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.3487 - val_loss: 1.6840
Epoch 68/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.3466 - val_loss: 1.6257
Epoch 69/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.3443 - val_loss: 1.6637
Epoch 70/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.3266 - val_loss: 1.6611
Epoch 71/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.3396 - val_loss: 1.6247
Epoch 72/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.3235 - val_loss: 1.6238
Epoch 73/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.3142 - val_loss: 1.6725
Epoch 74/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.3063 - val_loss: 1.6231
Epoch 75/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.3130 - val_loss: 1.6500
Epoch 76/3

26400/26400 [==============================] - 3s 103us/sample - loss: 1.1251 - val_loss: 1.5540
Epoch 142/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1274 - val_loss: 1.5140
Epoch 143/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.1214 - val_loss: 1.5327
Epoch 144/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.1174 - val_loss: 1.5419
Epoch 145/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.1184 - val_loss: 1.5370
Epoch 146/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.1187 - val_loss: 1.5342
Epoch 147/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.1137 - val_loss: 1.5950
Epoch 148/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.1294 - val_loss: 1.5082
Epoch 149/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1094 - val_loss: 1.5274
Epoch 150/300
2

Epoch 215/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0280 - val_loss: 1.5171
Epoch 216/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0295 - val_loss: 1.4738
Epoch 217/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0317 - val_loss: 1.4577
Epoch 218/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0257 - val_loss: 1.5015
Epoch 219/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.0348 - val_loss: 1.4608
Epoch 220/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0164 - val_loss: 1.4988
Epoch 221/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0257 - val_loss: 1.4529
Epoch 222/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0223 - val_loss: 1.5001
Epoch 223/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0208 - val_loss: 1.4530
E

Epoch 22/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.7371 - val_loss: 2.0320
Epoch 23/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.7124 - val_loss: 1.8487
Epoch 24/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.7002 - val_loss: 1.8548
Epoch 25/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.6837 - val_loss: 1.7840
Epoch 26/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.6838 - val_loss: 1.9763
Epoch 27/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.6648 - val_loss: 1.7940
Epoch 28/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.6530 - val_loss: 1.8169
Epoch 29/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.6275 - val_loss: 1.7923
Epoch 30/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.6200 - val_loss: 1.7479
Epoch 31/3

...

26400/26400 [==============================] - 3s 104us/sample - loss: 1.1528 - val_loss: 1.5651
Epoch 121/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1516 - val_loss: 1.5607
Epoch 122/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.1483 - val_loss: 1.5253
Epoch 123/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.1475 - val_loss: 1.6028
Epoch 124/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.1534 - val_loss: 1.5428
Epoch 125/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.1486 - val_loss: 1.6172
Epoch 126/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1423 - val_loss: 1.4798
Epoch 127/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1397 - val_loss: 1.5361
Epoch 128/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1403 - val_loss: 1.5393
Epoch 129/300
2

Epoch 194/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0358 - val_loss: 1.4920
Epoch 195/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0341 - val_loss: 1.5042
Epoch 196/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.0267 - val_loss: 1.4797
Epoch 197/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0235 - val_loss: 1.4772
Epoch 198/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0311 - val_loss: 1.4276
Epoch 199/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0320 - val_loss: 1.4581
Epoch 200/300
26400/26400 [==============================] - 3s 107us/sample - loss: 1.0172 - val_loss: 1.4360
Epoch 201/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0260 - val_loss: 1.5046
Epoch 202/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0195 - val_loss: 1.5160
E

Epoch 268/300
26400/26400 [==============================] - 3s 103us/sample - loss: 0.9582 - val_loss: 1.4855
Epoch 269/300
26400/26400 [==============================] - 3s 103us/sample - loss: 0.9607 - val_loss: 1.4988
Epoch 270/300
26400/26400 [==============================] - 3s 103us/sample - loss: 0.9621 - val_loss: 1.4829
Epoch 271/300
26400/26400 [==============================] - 3s 104us/sample - loss: 0.9674 - val_loss: 1.5070
Epoch 272/300
26400/26400 [==============================] - 3s 102us/sample - loss: 0.9486 - val_loss: 1.4438
Epoch 273/300
26400/26400 [==============================] - 3s 103us/sample - loss: 0.9644 - val_loss: 1.4443
Epoch 274/300
26400/26400 [==============================] - 3s 103us/sample - loss: 0.9578 - val_loss: 1.4600
Epoch 275/300
26400/26400 [==============================] - 3s 103us/sample - loss: 0.9638 - val_loss: 1.4231
Epoch 276/300
26400/26400 [==============================] - 3s 102us/sample - loss: 0.9598 - val_loss: 1.4672
E

26400/26400 [==============================] - 3s 103us/sample - loss: 1.1224 - val_loss: 1.5050
Epoch 136/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1093 - val_loss: 1.4655
Epoch 137/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.1001 - val_loss: 1.5506
Epoch 138/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1010 - val_loss: 1.5054
Epoch 139/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.1194 - val_loss: 1.5056
Epoch 140/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1026 - val_loss: 1.5196
Epoch 141/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.1041 - val_loss: 1.5030
Epoch 142/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0948 - val_loss: 1.5020
Epoch 143/300
26400/26400 [==============================] - 3s 105us/sample - loss: 1.1065 - val_loss: 1.4970
Epoch 144/300
2

Epoch 209/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.0166 - val_loss: 1.4731
Epoch 210/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0129 - val_loss: 1.4426
Epoch 211/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0158 - val_loss: 1.4439
Epoch 212/300
26400/26400 [==============================] - 3s 102us/sample - loss: 1.0095 - val_loss: 1.4113
Epoch 213/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0012 - val_loss: 1.4382
Epoch 214/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0129 - val_loss: 1.4717
Epoch 215/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0057 - val_loss: 1.4655
Epoch 216/300
26400/26400 [==============================] - 3s 103us/sample - loss: 1.0046 - val_loss: 1.4391
Epoch 217/300
26400/26400 [==============================] - 3s 104us/sample - loss: 1.0056 - val_loss: 1.4259
E

In [10]:
# 保存结果
today = str(datetime.date.today())
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
NN_saved=pd.DataFrame(NN_oof,columns=['oof'])
NN_saved.to_csv('nn_oof.csv',index=False)
submission["time_to_failure"] = NN_predictions
submission.to_csv(f'NN_submission_{cv_score:.3f}.csv', index=False)
submission.head()

,seg_id,time_to_failure
0,seg_00030f,3.248064
1,seg_0012b5,5.236308
2,seg_00184e,4.278768
3,seg_003339,7.400967
4,seg_0042cc,6.239193


In [11]:
print(cv_score)

1.4223067946969568
